# Criando uma App no Twitter

## Acessar o endereço abaixo e criar uma App: https://apps.twitter.com/

Criar login, senha e logar

Criar uma nova App clicando em Create New App

Definir os detalhes da aplicação: nome, descrição, website, etc

**No menu "Keys and Tokens" gerar as chaves da App para usar na configuração do Flume e substituir no arquivo twitterAgent.conf abaixo.**

# Configurando o agent, source, channel e sink

**Agent**: Apenas um agente chamado *TwitterAgent*

**Source**: Twitter

**Channel**: Memória

**Sink**: Registra os dados no HDFS

In [1]:
!cat /home/jovyan/labs/lab6-flume/twitterAgent.conf

#Nome dos componentes do agente
TwitterAgent.sources = Twitter
TwitterAgent.channels = MemChannel
TwitterAgent.sinks = HDFS

#Configuração do Source
TwitterAgent.sources.Twitter.type = com.cloudera.flume.source.TwitterSource
TwitterAgent.sources.Twitter.consumerKey = 6BWSmQX6AUfKhcNsMeav9zhi2
TwitterAgent.sources.Twitter.consumerSecret = DcYHM3EFR5oJR7VEq8cBVtjTPQxftI9PMrST71P7oXW0BlGiZv
TwitterAgent.sources.Twitter.accessToken = 1046705580-MTYNfMbLL6XSyQQgeL3Sah9RejwDRK5caBO9GRZ
TwitterAgent.sources.Twitter.accessTokenSecret = 2FRFyHQEdAIFVCFgTxyxCvl4zqoNtTEMZuwJHCfhXW2jk
TwitterAgent.sources.Twitter.keywords = #hadoop, #flume, #bigdata

#Configuração do Sink
TwitterAgent.sinks.HDFS.type = hdfs
TwitterAgent.sinks.HDFS.hdfs.path = hdfs://localhost:9000/user/matheus
TwitterAgent.sinks.HDFS.hdfs.fileType = DataStream
TwitterAgent.sinks.HDFS.hdfs.writeFormat = Text

#number of events written to file before it is flushed to HDFS
TwitterAgent.sinks.HDFS.hdfs.batchSize = 50 

#File size to t

## Nota: 
Usando as configurações descritas no site do Flume - **TwitterAgent.sources.Twitter.type = org.apache.flume.source.twitter.TwitterSource** - na configuração do source, o arquivo é gerado com caracteres ilegíveis. Assim, iremos utilizar **TwitterAgent.sources.Twitter.type = com.cloudera.flume.source.TwitterSource** e temos que copiar o arquivo flume-sources-1.0-SNAPSHOT.jar para a pasta do Flume para o seu correto funcionamento.

In [2]:
!cp resources/flume-sources-1.0-SNAPSHOT.jar ~/resources/local/flume-${FLUME_VERSION}/lib

# Abrir um terminal e iniciar o FlumeAgent

``` bash
flume-ng agent --conf conf --conf-file labs/lab6-flume/twitterAgent.conf --name TwitterAgent -Dflume.looger=INFO,console
``` 

# TrendTopics
Criando um rank de palavras que contém #

In [3]:
!python resources/mrjob-ex-3.py -r hadoop --hadoop-streaming-jar $HADOOP_HOME/share/hadoop/tools/lib/hadoop-streaming-2.9.2.jar hdfs:///user/matheus/*

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /home/jovyan/resources/local/hadoop-2.9.2/bin...
Found hadoop binary: /home/jovyan/resources/local/hadoop-2.9.2/bin/hadoop
Using Hadoop version 2.9.2
Creating temp directory /tmp/mrjob-ex-3.jovyan.20190913.175842.269968
uploading working dir files to hdfs:///user/jovyan/tmp/mrjob/mrjob-ex-3.jovyan.20190913.175842.269968/files/wd...
Copying other local files to hdfs:///user/jovyan/tmp/mrjob/mrjob-ex-3.jovyan.20190913.175842.269968/files/
Running step 1 of 2...
  packageJobJar: [/tmp/hadoop-unjar4483334449690778934/] [] /tmp/streamjob4300404582507966699.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input files to process : 4
  Cleaning up the staging area /tmp/hadoop-yarn/staging/jovyan/.staging/job_1568395552187_0004
  Error Launching job : Not a file: hdfs://localhost:9000/user/matheus/output